In [167]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext(master = "local", appName="trasformacionesYAcciones")

In [146]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [147]:
equiposOlimpicosRDD = sc.textFile('./files/paises.csv').map(lambda line: line.split(','))
equiposOlimpicosRDD.take(15)


[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [148]:
equiposOlimpicosRDD.map(lambda x : (x[2])).distinct().count()

231

In [149]:
equiposOlimpicosRDD.map(lambda x: (x[2],x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [150]:
equiposOlimpicosRDD.map(lambda x: (x[2],x[1])).groupByKey().mapValues(len).take(10)


[('sigla', 1),
 ('AUT', 11),
 ('MEX', 9),
 ('ARG', 18),
 ('AFG', 1),
 ('IRL', 7),
 ('SUI', 17),
 ('ALB', 1),
 ('POR', 21),
 ('FRA', 155)]

In [151]:
equipos_argentinos = equiposOlimpicosRDD.filter(lambda l : "ARG" in  l)
equipos_argentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [182]:
equiposOlimpicosRDD.count()

1185

In [153]:
equiposOlimpicosRDD.countApprox(20)

1185

In [168]:
deportistasOlimpicosRDD = sc.textFile("./files/deportista.csv").map(lambda l : l.split(','))
deportistasOlimpicosRDD2 = sc.textFile('./files/deportista2.csv').map(lambda l: l.split('/'))

deportistasOlimpicosRDD = deportistasOlimpicosRDD.union(deportistasOlimpicosRDD2)

In [169]:
deportistasOlimpicosRDD.count()

135572

In [170]:
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [171]:
deportistasOlimpicosRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999,Alexander Grant Alick Rennie,1,32,182,71,967']]

In [172]:
deportistaPaises = deportistasOlimpicosRDD\
                        .map(lambda l: [l[-1],l[:-1]])\
                        .join(equiposOlimpicosRDD.map(lambda x: [x[0],x[2]]))\
                        
                        

In [173]:
resultados = sc.textFile('./files/resultados.csv')\
    .map(lambda l: l.split(','))
resultados.take(5)


[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['1', 'NA', '1', '39', '1'],
 ['2', 'NA', '2', '49', '2'],
 ['3', 'NA', '3', '7', '3'],
 ['4', 'Gold', '4', '2', '4']]

In [174]:
resultadosGanador = resultados.filter(lambda l: "NA" not in l[1])
resultadosGanador.take(2)


[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [175]:
paisesMedallas = deportistaPaises.join(resultadosGanador)

In [176]:
valoresMedallas = {
    "Gold": 7,
    "Silver": 5,
    "Bronze": 4
}


In [177]:
paisesMedallas = paisesMedallas\
    .map(lambda x: [x[1][0][-1],valoresMedallas[x[1][1]]])

In [179]:
from operator import add
conclucion = paisesMedallas.reduceByKey((add))\
    .sortBy(lambda x : x[1], ascending=False)

In [181]:
conclucion.take(10)

[('CAN', 16571),
 ('ARG', 6454),
 ('HUN', 5636),
 ('MEX', 3028),
 ('RSA', 1740),
 ('BLR', 1540),
 ('MGL', 765),
 ('USA', 685),
 ('LTU', 610),
 ('AZE', 588)]